In [126]:
import pygame as pg
import numpy as np
from numpy import random
import pdb
import copy
from time import sleep
import pandas as pd



Game should start with 9 tiles

these 9 cards always contain at least one 3 and at most 4 of either color. (4 ones,4, twos, 1 three)

The highest number you can draw is 3 ranks below your high card. If your high card is 24 or less, you only draw 3s. With a high card of 48, 6s start to show up. At 96, 12s come in and at 192, 24s appear.


The probability for each possible bonus card value seems to be equal. This means that as your high card increases it is more and more unlikely that you will see the highest possible bonus card. On the flip side, the value of bonus cards (even if they are announced in the coming update) becomes less and less predictable as the high card increases.


• The game uses an undrawn stack of 12 cards (four 1s, four 2s and four 3s). A stack is created at the beginning of a game and cards are drawn individually both to fill the board (see above) and to be added with every move. When a stack is depleted, a new one is created. I know I posited this theory before and then retracted it but it just fits to many beginnings of sequences (where I suspect the error-rate is lower) and I still believe in it. This has some consequences:


• I am not sure when a new high card takes effect. It either does so immediately (during a running stack) or at the start of the next stack (see below on the topic of stacks). Figuring this out is particularly nasty due to the problems listed in section 1 and the very low probability that you see an appropriate bonus card after the high card increases but before a new stack begins.


It could just as well be possible that bonus cards are completely unrelated to stacks. Instead, there is just a constant 5 % chance on every move that your next card is a bonus card

In [161]:
current_score = []

In [193]:
class Tiles:
    def __init__(self):
        
        self.possible_icons  = np.array([1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144, 12288])
        self.tile_values  = np.array([0, 0, 3, 9, 27, 81, 243, 729, 2187, 6561, 19683, 59049, 177147, 531441, 1500000])
        self.score = 0;
        self.icons = dict()
        self.values = dict()
        pos = 0
        for index in self.possible_icons:
            
            self.icons[index] =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/_' + str(index) + '.png')
            self.values[index] =self.tile_values[pos]
            pos +=1
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
        self.stack = self.stack[random.permutation(9)]    
            
        self.grid = np.zeros((16,),dtype = int)
        self.grid[0:9] = self.stack
        self.grid =  np.reshape(self.grid,(4,4))
        self.move = np.zeros((4,4),dtype = bool)
        self.game_over = False
        
    def createStack(self):
        
        current_max = np.max(self.grid)
        
        max_position  = np.argwhere(self.possible_icons == current_max)
        bonus_cards = self.possible_icons[4:max_position[0][0]]
        
        self.stack = np.array([1,1,1,1,2,2,2,2,3])
        
        if bonus_cards.shape[0]>0:
            if random.rand(1)>.5:
                r =random.randint(0,len(bonus_cards))
                self.stack = np.append(self.stack,bonus_cards[r])
        rand_perm =  random.permutation(self.stack.shape[0])
        self.stack = self.stack[rand_perm]
        
        return self
        
    def drawStack(self):
        pos =0;
        for tile in self.stack:
            current_x = 400+ self.icons[tile].get_height()*np.floor(pos/4)
            current_y = self.icons[tile].get_height()*np.mod(pos,4)
            screen.blit(self.icons[tile],(current_x,current_y))
            
            pos +=1
                
            


    
    def determineMove(self,keys):
        current_move = [0, 0]
   
        if keys[pg.K_LEFT]:
            current_move = [0, -1]
  
        elif keys[pg.K_UP]:
            current_move = [-1, 0]
          
        elif keys[pg.K_RIGHT]:
            current_move = [0, 1]
            
        elif keys[pg.K_DOWN]:
            current_move = [1,0]
            
        return current_move
    def updatePosition(self,current_move):
        self.move = np.zeros((4,4),dtype = bool)
        x_range = np.array(range(4),dtype = int)
        y_range = np.array(range(4),dtype = int)
        if current_move[0] == 1:
            x_range = np.flip(x_range)
        if current_move[1] == 1:
            y_range = np.flip(y_range)
        
  
        for x in x_range:
            if x+current_move[0]>-1 and x+current_move[0]<4: #if the move does not take you off the grid
                for y in y_range:
                   
                    if y+current_move[1]>-1 and y+current_move[1]<4: #if the move does not take you off the grid
                        if self.grid[x,y] !=0: #if the current tile is not empty
                            new_pos = np.array([x,y]) + current_move #calculate where the current move will take this tile
                            if self.grid[new_pos[0],new_pos[1]] == 0: #if the new location is empty
                                self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y] #fill new location with tile
                                self.grid[x,y] = 0 #empty previous location
                                self.move[x,y] = True #record move
                                
                            elif self.grid[new_pos[0],new_pos[1]] + self.grid[x,y] == 3: #if move combines  1 and 2
                                self.grid[x,y] =0 #empty old position
                                self.grid[new_pos[0],new_pos[1]] =3 #put 3 in new position
                                self.move[x,y] = True
                               
                            elif self.grid[new_pos[0],new_pos[1]] == self.grid[x,y]:
                                if self.grid[new_pos[0],new_pos[1]] != 2:
                                    if self.grid[new_pos[0],new_pos[1]] + self.grid[x,y]>3:
                                    
                                        self.grid[new_pos[0],new_pos[1]]  = self.grid[x,y]*2
                                        self.grid[x,y]=0
                                        self.move[x,y] = True
                                    
                            

        return self
    
    def addTile(self,current_move):
        if np.sum(self.move,axis = None)>0:
      
            if current_move[0] ==-1:
                empty_spaces =  self.grid[3,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[0] ==1:
                empty_spaces =  self.grid[0,:]==0
                moved_spaces =  np.sum(self.move,axis = 0)

            elif current_move[1] ==-1:
                empty_spaces =  self.grid[:,3]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            elif current_move[1] ==1:
                empty_spaces =  self.grid[:,0]==0
                moved_spaces =  np.sum(self.move,axis = 1)

            new_spaces = empty_spaces * moved_spaces
            new_spaces = np.nonzero(new_spaces)
            new_spaces=new_spaces[0]

            if len(new_spaces)>0:
                r = np.random.permutation(len(new_spaces))
                new_spaces = new_spaces[r[0]]
                if current_move[0] == -1:
                    self.grid[3,new_spaces] = self.stack[0]
                elif current_move[0] == 1:
                    self.grid[0,new_spaces] = self.stack[0]
                elif current_move[1] == -1:
                    self.grid[new_spaces,3] = self.stack[0]
                elif current_move[1] == 1:
                    self.grid[new_spaces,0] = self.stack[0]
                  
                else:
                    print('No tile added!!!!!!!')
           
            else:
                pdb.set_trace()
                
            if self.stack.shape[0]>1:
                self.stack = np.delete(self.stack,0)
            else:
                print('created new stack')
                self =  self.createStack()
        else:
            self.game_over = True
            print('no move possible... end of game!')
            pdb.set_trace()
        return self
        
    def drawTile(self):
        for x in range(4):
            for y in range(4):
                if self.grid[x,y] !=0:
                    current_number = self.grid[x,y]
                    if current_number in self.icons:
                        screen.blit(self.icons[current_number],(y*self.icons[current_number].get_width(),
                                      x*self.icons[current_number].get_height()))
                    else:
                        print('Invalid current_number = ' + str(current_number))
                        running = False
                    
                    
    def calculateScore(self):
        self.score =0;
        for x in range(4):
            for y in range(4):
                current_number = self.grid[x,y]
                if current_number>0:
                    self.score = self.score + self.values[current_number]
        return self

        
        
   
    

In [200]:
class HiveMind:
    def __init__(self):
        self.current_move = np.array([0,0],dtype = int)
    def possibleMatchMoves(self,grid,current_move):
        temp_tiles = Tiles()
        temp_tiles.grid = copy.copy(grid)
        temp_tiles.calculateScore()
        current_score = temp_tiles.score
        temp_tiles.updatePosition(current_move)
        
        temp_tiles.calculateScore()
        new_score = temp_tiles.score
        score_change =  new_score - current_score
        possible_moves = np.sum(temp_tiles.move, axis = None)
     
        
        return possible_moves,score_change

In [197]:
def wait():
    loop = True
    while loop:
        for event in pg.event.get():
            if event.type == pg.KEYDOWN and event.key == pg.K_f:
                loop = False
            if event.type == pg.QUIT:
                loop = False
                running = False
                pg.display.quit()
    return

In [201]:
high_score_df = pd.read_pickle("high_score.pkl")

running = True
current_x = 0
current_y = 0
pg.init()
screen = pg.display.set_mode((800,600))
screen.fill((0,0,255))
pg.display.set_caption("Threes!")

# create a font object. 
# 1st parameter is the font file 
# which is present in pygame. 
# 2nd parameter is size of the font 
font = pg.font.Font('freesansbold.ttf', 50) 
  
# create a text suface object, 
# on which text is drawn on it. 
score_label = font.render('Score', True, (255,255,255), (0,0,0)) 
  
# create a rectangular object for the 
# text surface object 
textRect = score_label.get_rect()  
  
# set the center of the rectangular object.
textRect.center = (100,500) 

font = pg.font.Font('freesansbold.ttf', 100) 
game_over = font.render('Game Over :(', True, (255,255,255), (0,0,0)) 
gameOverRect = game_over.get_rect()  
gameOverRect.center = (400,300) 

grid  =pg.image.load('C:/Henry/PythonProjects/Pygame/Threes/images/grid.png')
tile_list = [1,2,3,6,12,24,48,96,192,384,768,1536,3072,6144]
start_num = 1

game_tiles = Tiles()

ex_state=True
hm = HiveMind()
PlayerName = 'HM'
score_change = np.zeros((4,)).astype(int) # score change for up, right, down, left
possible_move = np.zeros((4,)).astype(int) 
while running:
    #pg.time.delay(100)
    screen.fill((0,0,255))
    if game_tiles.game_over == False: 
        screen.blit(grid,(0,0))

        score_change[0],possible_move[0] = hm.possibleMatchMoves(game_tiles.grid,[-1, 0])
        score_change[1],possible_move[1] = hm.possibleMatchMoves(game_tiles.grid,[0, 1])
        score_change[2],possible_move[2] = hm.possibleMatchMoves(game_tiles.grid,[1,0])
        score_change[3],possible_move[3] = hm.possibleMatchMoves(game_tiles.grid,[0, -1])
        best_move_index = np.argwhere(score_change == np.max(score_change))
        most_possible_moves_index = np.argwhere(possible_move == np.max(possible_move))
        if np.max(score_change)!=0:
            if best_move_index.shape[0]==1:
                best_move = best_move_index[0][0]
            else:
                r = random.permutation(best_move_index.shape[0])
                best_move  = best_move_index[r[0]][0]

            if best_move == 0:
                print('i think I will move up')
                current_move = [-1, 0]
            elif best_move == 1:
                print('i think I will move right')
                current_move = [0, 1]
            elif best_move == 2:
                print('i think I will move down')
                current_move = [1, 0]
            elif best_move == 3:
                print('i think I will move left')
                current_move = [0, -1]
        else:
            if most_possible_moves_index.shape[0]==1:
                best_move = most_possible_moves_index[0][0]
            else:
                r = random.permutation(most_possible_moves_index.shape[0])
                best_move  = most_possible_moves_index[r[0]][0]

            if best_move == 0:
                print('i think I will move up')
                current_move = [-1, 0]
            elif best_move == 1:
                print('i think I will move right')
                current_move = [0, 1]
            elif best_move == 2:
                print('i think I will move down')
                current_move = [1, 0]
            elif best_move == 3:
                print('i think I will move left')
                current_move = [0, -1]
            

        game_tiles = game_tiles.updatePosition(current_move)
        game_tiles = game_tiles.addTile(current_move)




        game_tiles.drawTile()
        game_tiles.drawStack()
        game_tiles = game_tiles.calculateScore()
        game_score = font.render(str(game_tiles.score), True, (255,255,255), (0,0,0)) 
        scoreRect = game_score.get_rect()  
        scoreRect.center = (400,500) 
        screen.blit(score_label, textRect) 
        screen.blit(game_score, scoreRect) 

        font = pg.font.Font('freesansbold.ttf', 32) 
    else:
        screen.blit(game_over, gameOverRect) 
        
        
  
    
    
    
        
    
            
    pg.display.update()
    
    sleep(1)

    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False
            pg.display.quit()
        if event.type == pg.KEYUP:
            ex_state = True;


current_score = ([[PlayerName, game_tiles.score]])

current_score_df = pd.DataFrame(current_score, columns = ['Name', 'score']) 

high_score_df = high_score_df.append(current_score_df)

high_score_df.to_pickle('high_score.pkl')

    
      
            

i think I will move down
i think I will move up
i think I will move down
i think I will move up
i think I will move down
i think I will move up
i think I will move down
i think I will move up
i think I will move down
created new stack
i think I will move up
i think I will move down
i think I will move right
i think I will move up
i think I will move left
i think I will move down
i think I will move up
i think I will move down
i think I will move up
created new stack
i think I will move down
i think I will move up
i think I will move right
i think I will move down
i think I will move up
i think I will move down
i think I will move up
i think I will move down
i think I will move up
created new stack
i think I will move down
i think I will move up
i think I will move down
i think I will move left
i think I will move up
i think I will move right
i think I will move left
i think I will move right
i think I will move left
i think I will move right
created new stack
i think I will move down
i

In [199]:

pg.display.quit()



In [192]:

high_score_df

,Name,score
0,HM,909
0,HM,63
0,HM,54
0,HM,162


In [165]:
current_score
current_score_df = pd.DataFrame(current_score, columns = ['Name', 'score']) 
current_score_df.to_pickle('high_score.pkl')

In [179]:

high_score_df.to_pickle('high_score.pkl')
 

In [177]:
current_score_df

,Name,score
0,HM,453


In [229]:
copy_grid.grid[0,0] = -1
print(new_tiles.grid)
print(copy_grid.grid)



[[1 1 2 2]
 [3 2 2 1]
 [1 0 0 0]
 [0 0 0 0]]
[[-1  1  2  2]
 [ 3  2  2  1]
 [ 1  0  0  0]
 [ 0  0  0  0]]


In [192]:
x =1
temp_func(x)
print(x)

1


In [203]:
x = 1
y=x
y=y+1
print(x)
print(y)

1
2
